In [1]:

import json
import pprint

import requests


/Users/apabook/Desktop/ds-deployment/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


# Testing HL Api for PUSH transfer

---

## Consts

In [2]:
data_space_provider = "http://127.0.0.1:1234"
data_space_consumer = "http://127.0.0.1:1235"
context_broker = "http://fiware-orion:1026"
streaming_processor = "http://streaming-testing-service:1237"

catalog_id = ""
dataservice_id = ""
agreement_id = ""
agreement_pid = ""

## Setup Catalog, Dataservice and agreements

In [3]:
payload = {
    "foaf:homepage": "My catalog in Dataspace provider",
    "dct:title": "My catalog in Dataspace provider"
}
headers = {
    "Content-Type": "application/json",
}
url = data_space_provider + "/api/v1/catalogs"
response = requests.request("POST", url, headers=headers, data=json.dumps(payload))
response_as_json = response.json()
catalog_id = response_as_json["@id"]

pprint.pprint(response.json())

{'@context': 'https://w3id.org/dspace/2024/1/context.json',
 '@id': '4de7ed4f-3be1-410d-ae56-16dace6d19ea',
 '@type': 'dcat:Catalog',
 'dcat:dataset': [],
 'dcat:keyword': '',
 'dcat:service': [],
 'dcat:theme': '',
 'dct:conformsTo': None,
 'dct:creator': None,
 'dct:description': [],
 'dct:identifier': '4de7ed4f-3be1-410d-ae56-16dace6d19ea',
 'dct:issued': '2024-12-02T10:06:52.184171',
 'dct:modified': None,
 'dct:title': 'My catalog in Dataspace provider',
 'dspace:extraFields': None,
 'dspace:participantId': None,
 'foaf:homepage': 'My catalog in Dataspace provider',
 'odrl:hasPolicy': None}


In [4]:
suscription_payload = {
    "description": "Air Quality Unit Subscription",
    "subject": {
        "entities":[
            {"idPattern":".*","type":"AirQualityUnit"}],
        "condition": {
            "attrs":["CO"]
        }
    },
    "notification": {
        "http": {
            "url":"$data_url",
        },
    },
    "expires":"2040-01-01T14:00:00.00Z",
    "throttling":5
}
payload = {
    "dcat:endpointURL": context_broker + "/v2/subscriptions",
    "dcat:endpointDescription": json.dumps(suscription_payload),
}
headers = {
    "Content-Type": "application/json",
}
url = data_space_provider + "/api/v1/catalogs/" + catalog_id + "/data-services"
response = requests.request("POST", url, headers=headers, data=json.dumps(payload))
response_as_json = response.json()
dataservice_id = response_as_json["@id"]

pprint.pprint(response.json())

{'@context': 'https://w3id.org/dspace/2024/1/context.json',
 '@id': 'd17716f4-868e-4ba9-8212-44c1bfacd724',
 '@type': 'dcat:DataService',
 'dcat:endpointDescription': '{"description": "Air Quality Unit Subscription", '
                             '"subject": {"entities": [{"idPattern": ".*", '
                             '"type": "AirQualityUnit"}], "condition": '
                             '{"attrs": ["CO"]}}, "notification": {"http": '
                             '{"url": "$data_url"}}, "expires": '
                             '"2040-01-01T14:00:00.00Z", "throttling": 5}',
 'dcat:endpointURL': 'http://fiware-orion:1026/v2/subscriptions',
 'dcat:keyword': '',
 'dcat:theme': '',
 'dct:conformsTo': None,
 'dct:creator': None,
 'dct:description': [],
 'dct:identifier': 'd17716f4-868e-4ba9-8212-44c1bfacd724',
 'dct:issued': '2024-12-02T10:06:52.743854',
 'dct:modified': None,
 'dct:title': None,
 'dspace:extraFields': None,
 'odrl:hasPolicy': None}


In [5]:
payload = {
    "dataServiceId": dataservice_id
}
headers = {
    "Content-Type": "application/json",
}
url = data_space_provider + "/api/v1/agreements"
response = requests.request("POST", url, headers=headers, data=json.dumps(payload))
response_as_json = response.json()
agreement_id = response_as_json["agreement_id"]
agreement_pid = "urn:uuid:" + agreement_id

pprint.pprint(response.json())

{'agreement_id': 'eb4aa5e0-0676-4698-a70c-66eef679b107',
 'data_service_id': 'd17716f4-868e-4ba9-8212-44c1bfacd724',
 'identity': None,
 'identity_token': None}


## Setup transfer

In [6]:
url = data_space_consumer + "/api/v1/setup-transfer"

payload = ""
headers = {}
response = requests.request("POST", url, headers=headers, data=payload)
callbackAddress = response.json()["callbackAddress"]
callbackId = response.json()["callbackId"]
consumerPid = response.json()["consumerPid"]

pprint.pprint(response.json())

{'callbackAddress': 'http://ds-consumer:1235/928f5eed-54d9-4760-b477-67cfa03cf7af',
 'callbackId': '928f5eed-54d9-4760-b477-67cfa03cf7af',
 'consumerPid': 'urn:uuid:a311893f-586e-42db-a1f4-3ad99ed45368'}


## Request transfer

In [7]:
url = data_space_consumer + "/api/v1/request-transfer"

payload = json.dumps({
    "agreementId": agreement_pid,
    "format": "fiware+push",
    "dataAddress": {
        "@type": "dspace:DataAddress",
        "dspace:endpoint": "http://host.docker.internal:5000/data",
        "dspace:endpointType": "HTTP",
        "dspace:endpointProperties": []
    },
    "callbackAddress": callbackAddress,
    "callbackId": callbackId,
    "consumerPid": consumerPid
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

pprint.pprint(response.json())


{'consumerPid': 'urn:uuid:a311893f-586e-42db-a1f4-3ad99ed45368',
 'transferProcess': {'@context': 'https://w3id.org/dspace/2024/1/context.json',
                     '@type': 'dspace:TransferProcess',
                     'dspace:consumerPid': 'urn:uuid:a311893f-586e-42db-a1f4-3ad99ed45368',
                     'dspace:providerPid': 'urn:uuid:10e3eec5-bc4e-4611-a22d-a2e8958041f7',
                     'dspace:state': 'dspace:REQUESTED'}}


#### For seeing how the push service is working, please check the logs in the streaming-testing-service

## Server

In [ ]:
from flask import Flask, request, jsonify
import json
import time

app = Flask(__name__)

@app.route('/data', methods=['POST'])
def receive_data():
    data = json.loads(request.get_data())
    timestamp = int(time.time() * 1000)
    filename = f"data/fiware_stream_{timestamp}.json"
    with open(filename, 'w') as f:
        json.dump(data, f)

    return jsonify({"status": "Data received"}), 200

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.81:5000
Press CTRL+C to quit
127.0.0.1 - - [02/Dec/2024 11:06:59] "POST /data HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2024 11:07:04] "POST /data HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2024 11:07:09] "POST /data HTTP/1.1" 200 -
